In [21]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit

In [22]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [45]:
def couzin(agents, speeds, N, s, noise, rr=1, ro=2, ra=3):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir 

def bias(speeds, N, s, p, bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * p)
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * bias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir

    

def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [50]:
def simulate(N_steps, a, dt, N, width, height, s):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """

    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        ## MODEL
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        couzin(agents, speeds, N, s, 0.05,100,1900,2000)
        #bias(speeds, N, s, p, bias, weight):
        bias(speeds, N, s, 0.01, np.array([0.0,-1.0]) , 1)
        #couple_speeds(agents, speeds, a, s, N)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)
        #print(v_avg)
        #v_avg = get_v_avg(speeds)
        #time.sleep(0.02)
        stop = timeit.default_timer()
        print stop - start
    window.close()

simulate(2000, 0.1, 1, 50, 500, 500, 2)

0.0102641582489
0.00952196121216
0.0116720199585
0.0108721256256
0.0113191604614
0.00907492637634
0.00937819480896
0.010724067688
0.016233921051
0.0127410888672
0.0107560157776
0.0115239620209
0.00970578193665
0.00996994972229
0.010272026062
0.0128109455109
0.0116858482361
0.0175709724426
0.0123538970947
0.0106019973755
0.0144488811493
0.00931286811829
0.0105350017548
0.0116038322449
0.0121819972992
0.0190141201019
0.0126287937164
0.0114281177521
0.0109860897064
0.0130698680878
0.0099401473999
0.010372877121
0.00960898399353
0.00938105583191
0.00942897796631
0.00930905342102
0.0102128982544
0.0120129585266
0.013649225235
0.011561870575
0.00994515419006
0.00909900665283
0.0107500553131
0.0131599903107
0.0120770931244
0.0114731788635
0.0114710330963
0.010066986084
0.0106060504913
0.0116100311279
0.0119371414185
0.0111701488495
0.00997519493103
0.0101048946381
0.011344909668
0.0111129283905
0.0121359825134
0.0106530189514
0.0171270370483
0.0133719444275
0.0115990638733
0.0102870464325
0.0

0.0174810886383
0.012845993042
0.00992488861084
0.0108709335327
0.0109860897064
0.0115559101105
0.0122668743134
0.00937914848328
0.00975203514099
0.0112009048462
0.0103499889374
0.011127948761
0.00939607620239
0.0103080272675
0.010537147522
0.0117900371552
0.0119931697845
0.014132976532
0.0186688899994
0.0107061862946
0.011687040329
0.00984692573547
0.0100150108337
0.0105571746826
0.010115146637
0.011137008667
0.0095009803772
0.0101099014282
0.0106930732727
0.0117230415344
0.0152208805084
0.011106967926
0.0120921134949
0.011323928833
0.0101280212402
0.0104699134827
0.0105690956116
0.0110521316528
0.0116000175476
0.0101401805878
0.00955390930176
0.0103259086609
0.0113739967346
0.011146068573
0.0108649730682
0.0107941627502
0.0118651390076
0.00960898399353
0.00975704193115
0.0109190940857
0.0108258724213
0.0110638141632
0.00984191894531
0.0100431442261
0.0111010074615
0.0111510753632
0.0118231773376
0.0115559101105
0.0178120136261
0.0108830928802
0.0121290683746
0.0111892223358
0.0181319

0.0115389823914
0.0125830173492
0.0122301578522
0.0100700855255
0.0099949836731
0.0107581615448
0.0112118721008
0.011029958725
0.0117950439453
0.010143995285
0.0107820034027
0.0112240314484
0.0111839771271
0.0116899013519
0.0115320682526
0.0128228664398
0.0121929645538
0.0107748508453
0.0104382038116
0.0108201503754
0.0113930702209
0.0109729766846
0.0112490653992
0.01154088974
0.0104398727417
0.0109190940857
0.011647939682
0.011962890625
0.0107710361481
0.0124690532684
0.0149290561676
0.0116889476776
0.0118451118469
0.010617017746
0.0102319717407
0.0118889808655
0.0116238594055
0.0111708641052
0.0113730430603
0.0112109184265
0.010274887085
0.0107579231262
0.0112960338593
0.0138218402863
0.0113739967346
0.0118119716644
0.0123109817505
0.0119049549103
0.0098090171814
0.0106019973755
0.0114870071411
0.0113730430603
0.0111269950867
0.0115840435028
0.0103981494904
0.0102779865265
0.0110561847687
0.0115180015564
0.0149590969086
0.0110969543457
0.0110471248627
0.0123288631439
0.0104558467865


0.0131280422211
0.0137438774109
0.0135760307312
0.0118789672852
0.0129899978638
0.0103688240051
0.0101969242096
0.0112671852112
0.0114538669586
0.0117080211639
0.0116579532623
0.0121579170227
0.0100750923157
0.0101799964905
0.014664888382
0.0101990699768
0.0114319324493
0.0112290382385
0.0117018222809
0.0118520259857
0.0115659236908
0.010370016098
0.0106179714203
0.0105619430542
0.0113551616669
0.0123779773712
0.0108370780945
0.0114920139313
0.0137939453125
0.0125198364258
0.0124399662018
0.0122771263123
0.01060795784
0.0101130008698
0.0114769935608
0.0108201503754
0.0114049911499
0.0112769603729
0.0102210044861
0.0104188919067
0.01092004776
0.0112850666046
0.0111050605774
0.0155739784241
0.0110428333282
0.0124161243439
0.0113201141357
0.0102069377899
0.0109078884125
0.0113608837128
0.0126049518585
0.0112400054932
0.0114960670471
0.0112340450287
0.0105328559875
0.010803937912
0.0125558376312
0.010351896286
0.0131561756134
0.0113589763641
0.0109050273895
0.0118489265442
0.0109210014343
